<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Parabolik_Sar_Uzaklik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli paketlerin kurulumu
!pip install git+https://github.com/rongardF/tvdatafeed matplotlib openpyxl
!pip install tradingview-screener==2.5.0
!pip install dash-bootstrap-components tqdm ta statsmodels mplfinance dash

import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from ta.trend import PSARIndicator
from ta.momentum import RSIIndicator
from ta.trend import MACD
import warnings
from tabulate import tabulate
from tradingview_screener import get_all_symbols
from datetime import datetime

warnings.filterwarnings("ignore", category=FutureWarning)

# TradingView bağlantısı
tv = TvDatafeed()

# Kullanıcıdan SAR pozisyonunu seçmesi
def get_sar_position():
    print("SAR'ın fiyatla pozisyonunu seçin:")
    print("1: SAR fiyatın üstünde")
    print("2: SAR fiyatın altında")
    choice = input("Seçiminizi yapın (1 veya 2): ")
    if choice == "1":
        print("SAR fiyatın üstünde olacak şekilde tarama yapılacak.")
        return "above"
    elif choice == "2":
        print("SAR fiyatın altında olacak şekilde tarama yapılacak.")
        return "below"
    else:
        print("Geçersiz seçim. Varsayılan olarak SAR fiyatın altında seçildi.")
        return "below"

# Lucid SAR hesaplama fonksiyonu
def calculate_lucid_sar(data, start, increment, maximum):
    sar = data['low'].copy()
    factor = start
    max_factor = maximum
    for i in range(1, len(data)):
        sar[i] = sar[i - 1] + factor * (data['high'][i - 1] - sar[i - 1])
        if factor < max_factor:
            factor += increment
    return sar

# Parabolic SAR hesaplama fonksiyonu
def calculate_parabolic_sar(data, start, increment, maximum):
    psar = PSARIndicator(
        high=data['high'],
        low=data['low'],
        close=data['close'],
        step=increment,
        max_step=maximum
    )
    return psar.psar()

# RSI hesaplama fonksiyonu
def calculate_rsi(data, period=14):
    rsi = RSIIndicator(close=data['close'], window=period)
    return rsi.rsi()

# MACD hesaplama fonksiyonu
def calculate_macd(data):
    macd = MACD(close=data['close'], window_slow=26, window_fast=12, window_sign=9)
    macd_line = macd.macd()
    signal_line = macd.macd_signal()
    return macd_line, signal_line

# Sembolleri tarama ve analiz
def scan_symbols(symbols, interval, start, increment, maximum, sar_position):
    results = []
    # Fiyat ile SAR arasındaki mesafenin %10 ile %15 arasında olması isteniyor
    threshold_lower, threshold_upper = 0, 15
    for symbol in symbols:
        try:
            # Veriyi al
            data = tv.get_hist(symbol=symbol, exchange="BIST", interval=interval, n_bars=500)
            if data is None or data.empty:
                continue

            # En son kapanış fiyatı
            current_price = data['close'].iloc[-1]

            # SAR hesaplamaları
            data['lucid_sar'] = calculate_lucid_sar(data, start, increment, maximum)
            data['parabolic_sar'] = calculate_parabolic_sar(data, start, increment, maximum)

            # RSI ve MACD hesaplamaları
            data['rsi'] = calculate_rsi(data)
            macd_line, signal_line = calculate_macd(data)
            data['macd_line'] = macd_line
            data['signal_line'] = signal_line
            data['macd_action'] = np.where(data['macd_line'] > data['signal_line'], 'Al', 'Sat')

            # SAR mesafelerini % olarak hesapla
            data['lucid_sar_distance'] = np.where(data['lucid_sar'] != 0,
                                                  abs(current_price - data['lucid_sar']) / data['lucid_sar'] * 100,
                                                  np.nan)
            data['parabolic_sar_distance'] = np.where(data['parabolic_sar'] != 0,
                                                      abs(current_price - data['parabolic_sar']) / data['parabolic_sar'] * 100,
                                                      np.nan)

            # Fiyatın haftalık Parabolic SAR'a %10 ile %15 yaklaştığı durumları filtrele
            filtered = data.copy()
            if sar_position == "above":
                filtered = filtered[
                    (filtered['close'] < filtered['parabolic_sar']) &
                    (filtered['parabolic_sar_distance'].between(threshold_lower, threshold_upper))
                ]
            elif sar_position == "below":
                filtered = filtered[
                    (filtered['close'] > filtered['parabolic_sar']) &
                    (filtered['parabolic_sar_distance'].between(threshold_lower, threshold_upper))
                ]

            if not filtered.empty:
                last_row = filtered.iloc[-1]
                results.append({
                    "Sembol": symbol,
                    "Anlık Fiyat": current_price,
                    "Parabolic SAR": last_row['parabolic_sar'],
                    "Parabolic Mesafe (%)": last_row['parabolic_sar_distance'],
                    "Lucid SAR": last_row['lucid_sar'],
                    "Lucid Mesafe (%)": last_row['lucid_sar_distance'],
                    "RSI": last_row['rsi'],
                    "MACD Durumu": last_row['macd_action']
                })
        except Exception as e:
            print(f"{symbol} için hata: {e}")
    return results

if __name__ == "__main__":
    try:
        symbols = get_all_symbols(market='turkey')
        symbols = [symbol.replace("BIST:", "") for symbol in symbols]
    except Exception as e:
        print(f"Hata alındı: {e}")
        symbols = []

    # Haftalık veriler için ayar
    interval = Interval.in_weekly
    start, increment, maximum = 0.02, 0.02, 0.2

    sar_position = get_sar_position()
    results = scan_symbols(symbols, interval, start, increment, maximum, sar_position)

    if results:
        df_results = pd.DataFrame(results).sort_values(by=["Parabolic Mesafe (%)"], ascending=True)

        # Sonuçları Excel dosyasına kaydet
        file_name = f"BIST_SAR_Tarama_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df_results.to_excel(file_name, index=False)
        print(f"Sonuçlar Excel'e kaydedildi: {file_name}")

        # Tablo şeklinde çıktı
        print(tabulate(df_results, headers="keys", tablefmt="grid"))
    else:
        print("Uygun hisse bulunamadı.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-wf0gjy_7
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-wf0gjy_7
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


SAR'ın fiyatla pozisyonunu seçin:
1: SAR fiyatın üstünde
2: SAR fiyatın altında
Seçiminizi yapın (1 veya 2): 2
SAR fiyatın altında olacak şekilde tarama yapılacak.


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Sonuçlar Excel'e kaydedildi: BIST_SAR_Tarama_20250214_171334.xlsx
+-----+----------+---------------+-----------------+------------------------+--------------+--------------------+----------+---------------+
|     | Sembol   |   Anlık Fiyat |   Parabolic SAR |   Parabolic Mesafe (%) |    Lucid SAR |   Lucid Mesafe (%) |      RSI | MACD Durumu   |
+=====+==========+===============+=================+========================+==============+====================+==========+===============+
| 403 | BRMEN    |          4.95 |        4.95     |              0         |      6.10722 |         18.9484    |  46.6339 | Sat           |
+-----+----------+---------------+-----------------+------------------------+--------------+--------------------+----------+---------------+
| 134 | VKGYO    |          1.99 |        1.98965  |              0.0178105 |      2.16497 |          8.08202   |  54.7615 | Al            |
+-----+----------+---------------+-----------------+------------------------+-----------